In [ ]:
import anndata as ad
import h5py
import pandas as pd
import csv

In [2]:
data_path_dict = {
    "jurkat": "/work/jpma/Mouse_fasting_project/deyu/perturb_seq/data_with_deg/GSE264667_jurkat.h5ad",
    "hepg2": "/work/jpma/Mouse_fasting_project/deyu/perturb_seq/data_with_deg/GSE264667_hepg2.h5ad",
    "k562": "/work/jpma/Mouse_fasting_project/deyu/perturb_seq/data_with_deg/K562_essential.h5ad",
    "rpe1": "/work/jpma/Mouse_fasting_project/deyu/perturb_seq/data_with_deg/RPE1_essential.h5ad",
}


## 原始h5ad

In [ ]:
def byte2str(x):
    return x.decode("utf-8") if isinstance(x, (bytes, bytearray)) else str(x)

def print_list(name, ls):
    print(name)
    print(len(ls))
    print(ls)

h5ad_path = "/work/home/cryoem666/xyf/temp/pycharm/state/gene_perturnb_state/data/replogle.h5ad"

data = h5py.File(h5ad_path, "r")

hvg_mask = data["/var/highly_variable"][:]
all_gene_names = data["var/gene_name_index"][:]
hvg_names_raw = all_gene_names[hvg_mask]
hvg_names = [byte2str(name) for name in hvg_names_raw]

with open("output.csv", mode="w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(in1) # 写入第一行
    writer.writerow(in2)

In [4]:
symbol_to_id_all = {}
pert_names_given_all = []

def filter_p_value(p_vals, symbol_to_id):

    missing_names = [name for name in hvg_names if name not in symbol_to_id]
    print_list('Symbols not having Ensembl_ID:', missing_names)

    hvg_ensembl_names = [symbol_to_id[name] for name in hvg_names if name in symbol_to_id]
    print_list('hvg_ensembl_names', hvg_ensembl_names)

    existing_hvg_names = p_vals.columns.intersection(hvg_ensembl_names)
    print_list('existing_hvg_names', existing_hvg_names)

    p_vals_filtered = p_vals[existing_hvg_names]
    print("p_vals_filtered")
    print(p_vals_filtered.shape)

def process_cell_line(cell_line):

    global pert_names_given_all
    
    data_path = data_path_dict[cell_line]
    adata = ad.read_h5ad(data_path)

    p_vals = adata.varm['DE_pval'].T

    print(f"==== 正在处理{cell_line} ====")
    print("==== adata.var 预览 ====")
    print(adata.var.head())

    # 创建映射字典：{Symbol: Ensembl_ID}
    symbol_to_id = pd.Series(adata.var['ensembl_id'].values, index=adata.var['gene_symbol']).to_dict()
    symbol_to_id_all.update(symbol_to_id)
    print_list('symbol_to_id', symbol_to_id)

    filter_p_value(p_vals, symbol_to_id)

    temp = p_vals.index.tolist() + pert_names_given_all
    pert_names_given_all = list(set(temp))
    print_list("pert_names_given_all", pert_names_given_all)

## jurkat

In [5]:
process_cell_line("jurkat")

==== 正在处理jurkat ====
==== adata.var 预览 ====
                 gene_name   chr    start      end           class strand  \
ENSG00000237491  LINC01409  chr1   778747   810065  gene_version10      +   
ENSG00000228794  LINC01128  chr1   825138   868202   gene_version9      +   
ENSG00000188976      NOC2L  chr1   944203   959309  gene_version11      -   
ENSG00000188290       HES4  chr1   998962  1000172  gene_version10      -   
ENSG00000187608      ISG15  chr1  1001138  1014540  gene_version10      +   

                 length  in_matrix      mean       std        cv      fano  \
ENSG00000237491   31318       True  0.155002  0.417462  2.693264  1.124335   
ENSG00000228794   43064       True  0.287165  0.572515  1.993682  1.141413   
ENSG00000188976   15106       True  0.939766  1.160949  1.235359  1.434190   
ENSG00000188290    1210       True  2.612689  3.524809  1.349112  4.755362   
ENSG00000187608   13402       True  3.091917  2.869021  0.927910  2.662194   

                 n_cells

## hepg2

In [6]:
process_cell_line("hepg2")

==== 正在处理hepg2 ====
==== adata.var 预览 ====
                 gene_name   chr    start      end           class strand  \
ENSG00000228794  LINC01128  chr1   825138   868202   gene_version9      +   
ENSG00000188976      NOC2L  chr1   944203   959309  gene_version11      -   
ENSG00000187583    PLEKHN1  chr1   966482   975865  gene_version11      +   
ENSG00000188290       HES4  chr1   998962  1000172  gene_version10      -   
ENSG00000187608      ISG15  chr1  1001138  1014540  gene_version10      +   

                 length  in_matrix      mean       std        cv       fano  \
ENSG00000228794   43064       True  0.187369  0.462334  2.467510   1.140814   
ENSG00000188976   15106       True  1.742970  1.967864  1.129029   2.221775   
ENSG00000187583    9383       True  0.164274  0.444831  2.707855   1.204539   
ENSG00000188290    1210       True  2.179681  2.977747  1.366139   4.068017   
ENSG00000187608   13402       True  1.701508  4.689206  2.755911  12.923033   

                 n_

## k562

In [7]:
process_cell_line("k562")

==== 正在处理k562 ====
==== adata.var 预览 ====
                  chr    start      end           class strand  length  \
ENSG00000188976  chr1   944203   959309  gene_version11      -   15106   
ENSG00000187961  chr1   960584   965719  gene_version14      +    5135   
ENSG00000188290  chr1   998962  1000172  gene_version10      -    1210   
ENSG00000187608  chr1  1001138  1014540  gene_version10      +   13402   
ENSG00000078808  chr1  1216908  1232067  gene_version17      -   15159   

                 in_matrix      mean       std        cv      fano  \
ENSG00000188976       True  1.975144  1.707837  0.864665  1.476706   
ENSG00000187961       True  0.119593  0.353702  2.957540  1.046089   
ENSG00000188290       True  0.249577  0.561933  2.251540  1.265214   
ENSG00000187608       True  0.377373  0.787623  2.087120  1.643865   
ENSG00000078808       True  1.099141  1.173795  1.067920  1.253519   

                         gene_id  n_cells hg19_symbol gene_symbol  \
ENSG00000188976  ENSG00

## rpe1

In [8]:
process_cell_line("rpe1")

==== 正在处理rpe1 ====
==== adata.var 预览 ====
                  chr    start      end           class strand  length  \
ENSG00000188976  chr1   944203   959309  gene_version11      -   15106   
ENSG00000187583  chr1   966482   975865  gene_version11      +    9383   
ENSG00000188290  chr1   998962  1000172  gene_version10      -    1210   
ENSG00000187608  chr1  1001138  1014540  gene_version10      +   13402   
ENSG00000188157  chr1  1020120  1056118  gene_version15      +   35998   

                 in_matrix      mean       std        cv      fano  \
ENSG00000188976       True  0.997140  1.149519  1.152816  1.325185   
ENSG00000187583       True  0.131328  0.377933  2.877783  1.087609   
ENSG00000188290       True  0.732129  1.526109  2.084482  3.181148   
ENSG00000187608       True  0.455956  1.260789  2.765152  3.486272   
ENSG00000188157       True  0.346108  0.648378  1.873341  1.214633   

                         gene_id  n_cells hg19_symbol gene_symbol  \
ENSG00000188976  ENSG00

## ALL

In [9]:
print("==== replogle.h5ad 原数据 ====")
print(all_gene_names.shape)
print(all_gene_names)
print_list("hvg_names", hvg_names)

pert_names_raw = data["obs/gene_id/categories"][:2023]
pert_names = [byte2str(name) for name in pert_names_raw]
print_list("pert_names", pert_names)

data_path = data_path_dict["jurkat"]
adata = ad.read_h5ad(data_path)
p_vals = adata.varm['DE_pval'].T

filter_p_value(p_vals, symbol_to_id_all)

missing_names = [name for name in pert_names_given_all if name not in p_vals.index.tolist()]
print('Pert not included:')
print(len(missing_names))
print(missing_names)

==== replogle.h5ad 原数据 ====
(6642,)
[b'NOC2L' b'HES4' b'ISG15' ... b'MT-ND5' b'MT-ND6' b'MT-CYB']
hvg_names
2000
['HES4', 'ISG15', 'MIB2', 'CEP104', 'RERE', 'ENO1', 'SLC25A33', 'CTNNBIP1', 'KIF1B', 'KIAA2013', 'PRDM2', 'SPEN', 'NBPF1', 'RCC2', 'CAMK2N1', 'DDOST', 'RPL11', 'LYPLA2', 'GALE', 'HMGCL', 'FUCA1', 'NIPAL3', 'CLIC4', 'RSRP1', 'STMN1', 'SH3BGRL3', 'HMGN2', 'ARID1A', 'FAM76A', 'RPA2', 'SNHG3', 'TRNAU1AP', 'SNHG12', 'GMEB1', 'EPB41', 'PTP4A2', 'HDAC1', 'MARCKSL1', 'BSDC1', 'ZBTB8A', 'RBBP4', 'YARS', 'S100PBP', 'AK2', 'ZMYM6', 'SFPQ', 'ZMYM4', 'CLSPN', 'AGO1', 'AGO3', 'MRPS15', 'SNIP1', 'C1orf109', 'CDCA8', 'YRDC', 'FHL3', 'RRAGC', 'MYCBP', 'AKIRIN1', 'MACF1', 'PPIE', 'TRIT1', 'RLF', 'SMAP2', 'YBX1', 'SLC2A1', 'CDC20', 'KIF2C', 'RPS8', 'UROD', 'PRDX1', 'NASP', 'UQCRH', 'EFCAB14', 'STIL', 'FAF1', 'RNF11', 'ORC1', 'PRPF38A', 'MAGOH', 'LRP8', 'SSBP3', 'DHCR24', 'MYSM1', 'JUN', 'NFIA', 'USP1', 'DOCK7', 'GADD45A', 'ACADM', 'SSX2IP', 'BCL10', 'HS2ST1', 'LMO4', 'ZNF644', 'RPAP2', 'MTF2',

Symbols not having Ensembl_ID:
64
['YARS', 'C1orf109', 'HIST2H2AC', 'H3F3A', 'FAM126B', 'H1FX', 'TCTEX1D2', 'AC093323.1', 'H2AFZ', 'C4orf3', 'KIAA1109', 'TMEM161B-AS1', 'LINC01184', 'HIST1H1C', 'HIST1H4C', 'HIST1H2AC', 'HIST1H1E', 'HIST1H1B', 'ZNRD1', 'SNHG32', 'CASP8AP2', 'SOD2', 'FAM126A', 'GARS-DT', 'POLR2J3', 'WDR60', 'AC100810.1', 'MFHAS1', 'TNKS', 'FAM92A', 'CBWD5', 'AL158152.1', 'MFSD14C', 'AL441992.1', 'KIF1BP', 'RPARP-AS1', 'AP002360.1', 'CCDC84', 'H2AFX', 'ARNTL2', 'MARCH9', 'GIHCG', 'AC025159.1', 'C12orf49', 'C12orf65', 'CENPJ', 'ELMSAN1', 'WARS', 'NSMCE3', 'ADAL', 'LINC01578', 'SLC9A3R2', 'MARF1', 'ZNF720', 'AC004148.1', 'TMEM99', 'AC005670.3', 'SLC9A3R1', 'SEPTIN9', 'CCDC130', 'AC005261.1', 'CU638689.4', 'Z95331.1', 'TAZ']
hvg_ensembl_names
1936
['ENSG00000188290', 'ENSG00000187608', 'ENSG00000197530', 'ENSG00000116198', 'ENSG00000142599', 'ENSG00000074800', 'ENSG00000171612', 'ENSG00000178585', 'ENSG00000054523', 'ENSG00000116685', 'ENSG00000116731', 'ENSG00000065526', 'E